# Web Scraping do eSAJ do TJSP (2º Grau)

### Visão geral
Este programa automatiza a consulta ao eSAJ do TJSP (2º Grau) e organiza o resultado em dataframes, arquivo texto e planilha.

Os números dos processos são extraídos do arquivo "atos.csv" (gerado na área de intimação do próprio e-SAJ) ou de arquivo "txt", ainda que esses arquivos contenham outras informações.

### Questão ética
O TJSP não proíbe a raspagem de seu site (cf. em <https://esaj.tjsp.jus.br/robots.txt>). Evite, no entanto, reduzir o tempo das requisições. Prefira rodar o programa fora do expediente ou nos finais de semana.

In [1]:
import requests
import pandas as pd
import os
import re
import time
from datetime import datetime
from bs4 import BeautifulSoup
from tkinter import filedialog

In [2]:
data_e_hora_em_texto = datetime.now().strftime('%Y-%m-%d_%Hh%Mmin')

# Criação das listas vazias
lista_consulta = []
lista_resultados = []
lista_erros = []
lista_inconclusivos = []
lista_arquivos = []

In [3]:
# Opcionalmente, os números dos processos a serem pesquisados podem ser obtidos a partir dos nomes dos arquivos de uma pasta
# Função não implementada
def relaciona_processos(path):
    """Relaciona os números dos processos existentes na pasta"""
    for arquivo in os.listdir(path):
        lista_consulta.append(arquivo)

In [4]:
def encontra_processos (linha_de_texto):
    """Encontra os números de processos (CNJ) únicos nas linhas de texto examinadas."""
    resultado = re.findall(r'[0-9]{7}[-][0-9]{2}[.][0-9]{4}[.][8][.][2][6][.][0-9]{4}', 
                           linha_de_texto)
    for r in resultado:
        if r not in lista_arquivos:
            lista_arquivos.append(r) 
            
def ler_arquivo(path_do_arquivo):
    file = open(path_do_arquivo)
    for line in file:
       encontra_processos(line)
    print(f'Encontrei {len(lista_arquivos)} processos únicos.')
    return lista_arquivos

In [5]:
def pesquisa_processo(num_proc):
    """Retorna o html (content) da pesquisa"""
    params = (
    ('conversationId', ''),
    ('paginaConsulta', '0'),
    ('cbPesquisa', 'NUMPROC'),
    ('numeroDigitoAnoUnificado', num_proc),
    ('foroNumeroUnificado', num_proc[-4:]),
    ('dePesquisaNuUnificado', [num_proc, 'UNIFICADO']),
    ('dePesquisa', ''),
    ('tipoNuProcesso', 'UNIFICADO'),)
    return requests.get('https://esaj.tjsp.jus.br/cposg/search.do', params=params).content

In [6]:
def separa_dados(resultado):
    lista=[]
    for n in resultado:
        lista.append(n.text.strip())
    return lista

def extrai_dados(lista_consulta):
    for n_processo in lista_consulta: 
        html = pesquisa_processo(n_processo)
        soup = BeautifulSoup(html, 'html.parser')
        time.sleep(0.2)

        try:
            msg = soup.find(id='mensagemRetorno').text.strip()
            if msg:
                lista_erros.append([n_processo, msg])
                arquivo = open('nao_encontrados.txt', 'a')
                arquivo.write(n_processo + ' - ' + msg + ' - ' + data_e_hora_em_texto + '\n')
                arquivo.close
                
        except:
            try:
                # Número do processo
                numero = soup.find(id='numeroProcesso').text.strip()
                
                # Órgão julgador 
                orgao = soup.find(id='orgaoJulgadorProcesso').text.strip()
                
                # Relator do processo
                relator = soup.find(id='relatorProcesso').text.strip()
                
                # Classe do processo
                classe = soup.find(id='classeProcesso').text.strip()
                
                # Assunto
                assunto = soup.find(id='assuntoProcesso').text.strip()
                
                # Situação do processo
                situacao = soup.find(id='situacaoProcesso').text.strip()
                
                # Parte e advogado
                parte = soup.find(class_='nomeParteEAdvogado').text.strip()
                parte = parte.replace('\n', '')
                parte = parte.replace('\t', '')
                parte = parte.replace('  ', '')
                
                # Resultado final
                resultado =  soup.find_all('table')[-1].find_all('td')
                
                # Inclusão na lista de resultados
                lista_resultados.append([numero, orgao, relator, classe, 
                                         assunto, situacao, parte, separa_dados(resultado)])     

            except:
                try:
                    paginacao = soup.find(class_='resultadoPaginacao').text.strip()
                    lista_inconclusivos.append([n_processo, paginacao])
                    arquivo = open('inconclusivos.txt', 'a')
                    arquivo.write(n_processo + ' - ' + paginacao + ' - ' + data_e_hora_em_texto + '\n')
                    arquivo.close
                    
                except:
                    arquivo = open('nao_encontrados.txt', 'a')
                    arquivo.write(n_processo + ' - ' + data_e_hora_em_texto + '\n')
                    arquivo.close                

In [7]:
# Pesquisa por 'atos.csv' ou equivalente
ano = input('Entre com o ano de referência: ')
nome_pj = input('Entre com o nome do Procurador/PJ/Grupo/Vara: ')

file = filedialog.askopenfilename()
lista_arquivos = ler_arquivo(file)
extrai_dados(lista_arquivos)

Entre com o ano de referência: 2020-2021
Entre com o nome do Procurador/PJ/Grupo/Vara: 1a. Vara Criminal de Piracicaba
Encontrei 3281 processos únicos.


In [8]:
columns = ['Número do processo', 'Órgão Julgador', 'Relator', 'Classe',
           'Assunto', 'Situação', 'Recorrente(s)', 'Desfecho']
df = pd.DataFrame(lista_resultados, columns = columns)
df['Data'] = df['Desfecho'].str[0]
df['Status'] = df['Desfecho'].str[1]
df['Resultado'] = df['Desfecho'].str[2]
df = df.drop(columns='Desfecho')
df.head(3)

,Número do processo,Órgão Julgador,Relator,Classe,Assunto,Situação,Recorrente(s),Data,Status,Resultado
0,0009060-75.2018.8.26.0451,12ª Câmara de Direito Criminal,JOÃO MORENGHI,Apelação Criminal,DIREITO PENAL-Crimes contra o Patrimônio-Roubo,Encerrado,FERNANDO OLIDIO GIMENES Advogado: Luiz Antonio...,21/09/2021,Julgado,Negaram provimento ao recurso. V. U.
1,2249205-48.2021.8.26.0000,2ª Câmara de Direito Criminal,ANDRÉ CARVALHO E SILVA DE ALMEIDA,Habeas Corpus Criminal,DIREITO PENAL - Crimes Previstos na Legislação...,Julgado,Wesley Limongi Advogado: Sebastiao Zinsly,06/01/2022,Julgado,Denegaram a ordem. V. U.
2,1500712-64.2020.8.26.0599,5ª Câmara de Direito Criminal,CLAUDIA FONSECA FANUCCHI,Apelação Criminal,DIREITO PENAL - Crimes Previstos na Legislação...,Encerrado,Ministério Público do Estado de São Paulo,14/12/2020,Julgado,DERAM PROVIMENTO ao recurso do Ministério Públ...


In [9]:
# Processos em "segredo de justiça" ou não existentes em 2o. Grau
columns = ['Número do processo', 'Informação']
df_erros = pd.DataFrame(lista_erros, columns=columns)
df_erros.head(3)

,Número do processo,Informação
0,0008158-59.2017.8.26.0451,Não existem informações disponíveis para os pa...
1,1501634-71.2021.8.26.0599,Não existem informações disponíveis para os pa...
2,1502868-47.2021.8.26.0451,Não existem informações disponíveis para os pa...


In [10]:
columns = ['Número do processo', 'Observações']
df_inconclusivos = pd.DataFrame(lista_inconclusivos, columns=columns)
df_inconclusivos.head(3)

,Número do processo,Observações
0,1500748-72.2021.8.26.0599,2 Processos encontrados
1,1501611-28.2021.8.26.0599,3 Processos encontrados
2,1501801-88.2021.8.26.0599,3 Processos encontrados


In [11]:
# Criação de uma planilha Excel com o resultado do trabalho
with pd.ExcelWriter(f'{nome_pj}_{ano}_resultado_dos_recursos_{data_e_hora_em_texto}.xlsx') as writer:  
  df.to_excel(writer, sheet_name='resultados')
  df_erros.to_excel (writer, sheet_name='erros ou não processados')
  df_inconclusivos.to_excel(writer, sheet_name='inconclusivos')

In [12]:
# criação de um arquivo texto com o resultado do trabalho
with open(f'{nome_pj}_{ano}_resultado_dos_recursos_{data_e_hora_em_texto}.txt', 'w') as arquivo:
    arquivo.write (f'Resultado dos processos recebidos pelo [Procurador/PJ/Grupo/Vara] {nome_pj} no ano {ano} julgados pelo TJSP\n\n\n')
    for l in lista_resultados:
        arquivo.write (f'\nNúmero do processo: {l[0]}\n')
        arquivo.write (f'Órgão Julgador: {l[1]}\n')
        arquivo.write (f'Relator: {l[2]}\n')
        arquivo.write (f'Classe: {l[3]}\n')
        arquivo.write (f'Assunto: {l[4]}\n')
        arquivo.write (f'Situação: {l[5]}\n')
        arquivo.write (f'Recorrente: {l[6]}\n')
        try:
            arquivo.write (f'Data: {l[7][0]}\n')
            arquivo.write (f'Status: {l[7][1]}\n')
            arquivo.write (f'Resultado: {l[7][2]}\n\n')
            arquivo.write ('*' * 40 +'\n')
        except:
            arquivo.write ('*' * 40 +'\n')
    arquivo.write ('\n\n\nRelatório emitido em: '+ data_e_hora_em_texto)
    arquivo.write ('\nDesenvolvido em Python por @jespimentel')